### Bibliotecas ###

In [1]:
# Basics
import pandas as pd 
import numpy as np 
import io
from datetime import datetime, timedelta

# Visualization
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

# Others
import openpyxl
import xlrd
from pathlib import Path
import os
import warnings
from math import isnan
import category_encoders as ce

In [2]:
import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning) 
warnings.filterwarnings("ignore", category = FutureWarning) 
warnings.filterwarnings("ignore", category = UserWarning) 

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Import Dataset ###

In [4]:
# Dados dos empregados com vinculo ativo em 31/12 apenas da industria da transformação do ano de 2021
df_ind_trans = pd.read_csv(r"RAIS_VINC_PUB_MG_ES_RJ_Vinculo_0.csv", sep=";", encoding='UTF-8', low_memory=False)
df_ind_trans.shape

(1223051, 41)

In [5]:
df_ind_trans.head()

,CBO_Ocupação_2002,Vínculo_Ativo_31/12,Faixa_Etária,Faixa_Hora_Contrat,Faixa_Remun_Dezem_(SM),Faixa_Remun_Média_(SM),Faixa_Tempo_Emprego,Escolaridade_após_2005,Qtd_Hora_Contr,Idade,Ind_Simples,Município,Natureza_Jurídica,Ind_Portador_Defic,Qtd_Dias_Afastamento,Raça_Cor,Vl_Remun_Dezembro_Nom,Vl_Remun_Dezembro_(SM),Vl_Remun_Média_Nom,Vl_Remun_Média_(SM),CNAE_2.0_Subclasse,Sexo_Trabalhador,Tamanho_Estabelecimento,Tempo_Emprego,Tipo_Admissão,Tipo_Defic,Tipo_Vínculo,IBGE_Subsetor,Vl_Rem_Janeiro_SC,Vl_Rem_Fevereiro_SC,Vl_Rem_Março_SC,Vl_Rem_Abril_SC,Vl_Rem_Maio_SC,Vl_Rem_Junho_SC,Vl_Rem_Julho_SC,Vl_Rem_Agosto_SC,Vl_Rem_Setembro_SC,Vl_Rem_Outubro_SC,Vl_Rem_Novembro_SC,Ind_Trab_Intermitente,Ind_Trab_Parcial
0,782310,1,7,6,3,2,3,1,44,50,1,314800,2062,0,0,4,"0000001372,73","000001,24","0000001349,43","000001,22",3101200,1,4,"7,3",2,0,10,7,"000000000,00","000000000,00","000000000,00","000000000,00","000000005,00","000000015,00","000000015,00","000000015,00","000000015,00","000000015,00","000001422,73",0,0
1,821205,1,7,1,3,3,6,1,1,61,0,316720,2305,0,20,2,"0000001340,19","000001,21","0000001707,27","000001,54",2411300,1,7,"37,6",0,0,10,3,"000001672,64","000001563,77","000001540,25","000001769,53","000001664,24","000001660,49","000002246,68","000001565,25","000001662,41","000001845,41","000001956,40",0,0
2,821205,1,8,5,5,3,6,1,36,68,0,316720,2305,0,62,8,"0000002263,80","000002,05","0000001813,91","000001,64",2411300,1,7,"38,9",0,0,10,3,"000000019,43","000001815,11","000001816,52","000001820,07","000001932,60","000001929,03","000000522,66","000000888,54","000002467,33","000002122,71","000002245,57",0,0
3,811770,1,4,6,4,2,3,1,44,28,1,314520,2135,0,0,2,"0000002083,36","000001,89","0000001630,30","000001,48",1533500,1,3,"6,1",2,0,10,12,"000000000,00","000000000,00","000000000,00","000000000,00","000000000,00","000000358,80","000000017,94","000000017,94","000000017,94","000000017,94","000000017,94",0,0
4,514320,1,7,6,3,1,3,1,44,64,0,314110,2062,0,172,4,"0000001417,42","000001,28","0000001045,59","000000,94",2930101,1,6,"9,6",2,0,10,6,"000000000,00","000000000,00","000000927,08","000001324,40","000001368,55","000000838,79","000000000,00","000000000,00","000000000,00","000000000,00","000000397,32",0,0


In [6]:
str_cols = ['Vl_Rem_Janeiro_SC', 'Vl_Rem_Fevereiro_SC', 'Vl_Rem_Março_SC', 'Vl_Rem_Abril_SC', 'Vl_Rem_Maio_SC', 'Vl_Rem_Junho_SC', 
            'Vl_Rem_Julho_SC', 'Vl_Rem_Agosto_SC', 'Vl_Rem_Setembro_SC', 'Vl_Rem_Outubro_SC', 'Vl_Rem_Novembro_SC', 'Vl_Remun_Dezembro_Nom']   
df_ind_trans[str_cols] = df_ind_trans[str_cols].replace(',', '.', regex=True)
df_ind_trans[str_cols] = df_ind_trans[str_cols].astype(float)

In [7]:
df_ind_trans['N_0'] = df_ind_trans[['Vl_Rem_Janeiro_SC', 'Vl_Rem_Fevereiro_SC', 'Vl_Rem_Março_SC', 'Vl_Rem_Abril_SC', 'Vl_Rem_Maio_SC', 'Vl_Rem_Junho_SC', 
            'Vl_Rem_Julho_SC', 'Vl_Rem_Agosto_SC', 'Vl_Rem_Setembro_SC', 'Vl_Rem_Outubro_SC', 'Vl_Rem_Novembro_SC', 'Vl_Remun_Dezembro_Nom']].isin([0.0]).sum(axis=1)

In [8]:
df_ind_trans['N_0'].describe()

count   1223051.000
mean          2.421
std           3.819
min           0.000
25%           0.000
50%           0.000
75%           4.000
max          12.000
Name: N_0, dtype: float64

In [9]:
df_ind_trans = df_ind_trans.eval('Vlr_Rem_Ano = Vl_Rem_Janeiro_SC + Vl_Rem_Fevereiro_SC + Vl_Rem_Março_SC + Vl_Rem_Abril_SC + Vl_Rem_Maio_SC + Vl_Rem_Junho_SC + Vl_Rem_Julho_SC + Vl_Rem_Agosto_SC + Vl_Rem_Setembro_SC + Vl_Rem_Outubro_SC + Vl_Rem_Novembro_SC + Vl_Remun_Dezembro_Nom')

In [10]:
str_cols = ['Vl_Remun_Dezembro_(SM)', 'Vl_Remun_Média_Nom', 'Vl_Remun_Média_(SM)', 'Tempo_Emprego']   
df_ind_trans[str_cols] = df_ind_trans[str_cols].replace(',', '.', regex=True)
df_ind_trans[str_cols] = df_ind_trans[str_cols].astype(float)

In [11]:
df_ind_trans.isnull().sum()

CBO_Ocupação_2002          0
Vínculo_Ativo_31/12        0
Faixa_Etária               0
Faixa_Hora_Contrat         0
Faixa_Remun_Dezem_(SM)     0
Faixa_Remun_Média_(SM)     0
Faixa_Tempo_Emprego        0
Escolaridade_após_2005     0
Qtd_Hora_Contr             0
Idade                      0
Ind_Simples                0
Município                  0
Natureza_Jurídica          0
Ind_Portador_Defic         0
Qtd_Dias_Afastamento       0
Raça_Cor                   0
Vl_Remun_Dezembro_Nom      0
Vl_Remun_Dezembro_(SM)     0
Vl_Remun_Média_Nom         0
Vl_Remun_Média_(SM)        0
CNAE_2.0_Subclasse         0
Sexo_Trabalhador           0
Tamanho_Estabelecimento    0
Tempo_Emprego              0
Tipo_Admissão              0
Tipo_Defic                 0
Tipo_Vínculo               0
IBGE_Subsetor              0
Vl_Rem_Janeiro_SC          0
Vl_Rem_Fevereiro_SC        0
Vl_Rem_Março_SC            0
Vl_Rem_Abril_SC            0
Vl_Rem_Maio_SC             0
Vl_Rem_Junho_SC            0
Vl_Rem_Julho_S

In [12]:
df_ind_trans.columns

Index(['CBO_Ocupação_2002', 'Vínculo_Ativo_31/12', 'Faixa_Etária',
       'Faixa_Hora_Contrat', 'Faixa_Remun_Dezem_(SM)',
       'Faixa_Remun_Média_(SM)', 'Faixa_Tempo_Emprego',
       'Escolaridade_após_2005', 'Qtd_Hora_Contr', 'Idade', 'Ind_Simples',
       'Município', 'Natureza_Jurídica', 'Ind_Portador_Defic',
       'Qtd_Dias_Afastamento', 'Raça_Cor', 'Vl_Remun_Dezembro_Nom',
       'Vl_Remun_Dezembro_(SM)', 'Vl_Remun_Média_Nom', 'Vl_Remun_Média_(SM)',
       'CNAE_2.0_Subclasse', 'Sexo_Trabalhador', 'Tamanho_Estabelecimento',
       'Tempo_Emprego', 'Tipo_Admissão', 'Tipo_Defic', 'Tipo_Vínculo',
       'IBGE_Subsetor', 'Vl_Rem_Janeiro_SC', 'Vl_Rem_Fevereiro_SC',
       'Vl_Rem_Março_SC', 'Vl_Rem_Abril_SC', 'Vl_Rem_Maio_SC',
       'Vl_Rem_Junho_SC', 'Vl_Rem_Julho_SC', 'Vl_Rem_Agosto_SC',
       'Vl_Rem_Setembro_SC', 'Vl_Rem_Outubro_SC', 'Vl_Rem_Novembro_SC',
       'Ind_Trab_Intermitente', 'Ind_Trab_Parcial', 'N_0', 'Vlr_Rem_Ano'],
      dtype='object')

In [13]:
df_ind_trans = df_ind_trans[['CBO_Ocupação_2002', 'Faixa_Etária',
       'Faixa_Hora_Contrat', 'Faixa_Remun_Dezem_(SM)',
       'Faixa_Remun_Média_(SM)', 'Faixa_Tempo_Emprego',
       'Escolaridade_após_2005', 'Qtd_Hora_Contr', 'Idade', 'Ind_Simples',
       'Município', 'Natureza_Jurídica', 'Ind_Portador_Defic',
       'Qtd_Dias_Afastamento', 'Raça_Cor', 'Vl_Remun_Dezembro_Nom',
       'Vl_Remun_Dezembro_(SM)', 'Vl_Remun_Média_Nom', 'Vl_Remun_Média_(SM)',
       'CNAE_2.0_Subclasse', 'Sexo_Trabalhador', 'Tamanho_Estabelecimento',
       'Tempo_Emprego', 'Tipo_Admissão', 'Tipo_Defic', 'Tipo_Vínculo',
       'IBGE_Subsetor', 'Ind_Trab_Intermitente', 'Ind_Trab_Parcial', 'N_0', 'Vlr_Rem_Ano']]
df_ind_trans = df_ind_trans.reset_index()

In [14]:
df_ind_trans.head()

,index,CBO_Ocupação_2002,Faixa_Etária,Faixa_Hora_Contrat,Faixa_Remun_Dezem_(SM),Faixa_Remun_Média_(SM),Faixa_Tempo_Emprego,Escolaridade_após_2005,Qtd_Hora_Contr,Idade,Ind_Simples,Município,Natureza_Jurídica,Ind_Portador_Defic,Qtd_Dias_Afastamento,Raça_Cor,Vl_Remun_Dezembro_Nom,Vl_Remun_Dezembro_(SM),Vl_Remun_Média_Nom,Vl_Remun_Média_(SM),CNAE_2.0_Subclasse,Sexo_Trabalhador,Tamanho_Estabelecimento,Tempo_Emprego,Tipo_Admissão,Tipo_Defic,Tipo_Vínculo,IBGE_Subsetor,Ind_Trab_Intermitente,Ind_Trab_Parcial,N_0,Vlr_Rem_Ano
0,0,782310,7,6,3,2,3,1,44,50,1,314800,2062,0,0,4,1372.730,1.240,1349.430,1.220,3101200,1,4,7.300,2,0,10,7,0,0,4,2875.460
1,1,821205,7,1,3,3,6,1,1,61,0,316720,2305,0,20,2,1340.190,1.210,1707.270,1.540,2411300,1,7,37.600,0,0,10,3,0,0,0,20487.260
2,2,821205,8,5,5,3,6,1,36,68,0,316720,2305,0,62,8,2263.800,2.050,1813.910,1.640,2411300,1,7,38.900,0,0,10,3,0,0,0,19843.370
3,3,811770,4,6,4,2,3,1,44,28,1,314520,2135,0,0,2,2083.360,1.890,1630.300,1.480,1533500,1,3,6.100,2,0,10,12,0,0,5,2531.860
4,4,514320,7,6,3,1,3,1,44,64,0,314110,2062,0,172,4,1417.420,1.280,1045.590,0.940,2930101,1,6,9.600,2,0,10,6,0,0,6,6273.560


In [15]:
len(df_ind_trans[df_ind_trans['Vl_Remun_Dezembro_Nom'] == 0])

76164

In [16]:
len(df_ind_trans[df_ind_trans['Vlr_Rem_Ano'] == 0])

47106

In [17]:
df_ind_trans = df_ind_trans[df_ind_trans['Vlr_Rem_Ano'] > 0]
df_ind_trans = df_ind_trans[df_ind_trans['Vl_Remun_Dezembro_Nom'] > 0]
df_ind_trans.shape

(1146887, 32)

In [18]:
df_ind_trans[['Vlr_Rem_Ano', 'Vl_Remun_Dezembro_Nom']].describe()

,Vlr_Rem_Ano,Vl_Remun_Dezembro_Nom
count,1146887.000,1146887.000
mean,29559.267,3295.321
std,56801.797,5107.664
min,330.000,330.000
25%,7499.855,1441.540
50%,17721.400,1988.880
75%,30463.615,3182.850
max,8965544.090,163466.030


In [19]:
df_ind_trans['CBO_Ocupação_2002'] = df_ind_trans['CBO_Ocupação_2002'].replace('-', '', regex=True)
df_ind_trans['CBO_Ocupação_2002'] = df_ind_trans['CBO_Ocupação_2002'].astype(int)
df_ind_trans = df_ind_trans[df_ind_trans['CBO_Ocupação_2002'] != 1]
df_ind_trans.shape

(1146857, 32)

In [20]:
df_ind_trans.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1146857 entries, 0 to 1223050
Data columns (total 32 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   index                    1146857 non-null  int64  
 1   CBO_Ocupação_2002        1146857 non-null  int32  
 2   Faixa_Etária             1146857 non-null  int64  
 3   Faixa_Hora_Contrat       1146857 non-null  int64  
 4   Faixa_Remun_Dezem_(SM)   1146857 non-null  int64  
 5   Faixa_Remun_Média_(SM)   1146857 non-null  int64  
 6   Faixa_Tempo_Emprego      1146857 non-null  int64  
 7   Escolaridade_após_2005   1146857 non-null  int64  
 8   Qtd_Hora_Contr           1146857 non-null  int64  
 9   Idade                    1146857 non-null  int64  
 10  Ind_Simples              1146857 non-null  int64  
 11  Município                1146857 non-null  int64  
 12  Natureza_Jurídica        1146857 non-null  int64  
 13  Ind_Portador_Defic       1146857 non-null  int6

In [21]:
df_ind_trans.head()

,index,CBO_Ocupação_2002,Faixa_Etária,Faixa_Hora_Contrat,Faixa_Remun_Dezem_(SM),Faixa_Remun_Média_(SM),Faixa_Tempo_Emprego,Escolaridade_após_2005,Qtd_Hora_Contr,Idade,Ind_Simples,Município,Natureza_Jurídica,Ind_Portador_Defic,Qtd_Dias_Afastamento,Raça_Cor,Vl_Remun_Dezembro_Nom,Vl_Remun_Dezembro_(SM),Vl_Remun_Média_Nom,Vl_Remun_Média_(SM),CNAE_2.0_Subclasse,Sexo_Trabalhador,Tamanho_Estabelecimento,Tempo_Emprego,Tipo_Admissão,Tipo_Defic,Tipo_Vínculo,IBGE_Subsetor,Ind_Trab_Intermitente,Ind_Trab_Parcial,N_0,Vlr_Rem_Ano
0,0,782310,7,6,3,2,3,1,44,50,1,314800,2062,0,0,4,1372.730,1.240,1349.430,1.220,3101200,1,4,7.300,2,0,10,7,0,0,4,2875.460
1,1,821205,7,1,3,3,6,1,1,61,0,316720,2305,0,20,2,1340.190,1.210,1707.270,1.540,2411300,1,7,37.600,0,0,10,3,0,0,0,20487.260
2,2,821205,8,5,5,3,6,1,36,68,0,316720,2305,0,62,8,2263.800,2.050,1813.910,1.640,2411300,1,7,38.900,0,0,10,3,0,0,0,19843.370
3,3,811770,4,6,4,2,3,1,44,28,1,314520,2135,0,0,2,2083.360,1.890,1630.300,1.480,1533500,1,3,6.100,2,0,10,12,0,0,5,2531.860
4,4,514320,7,6,3,1,3,1,44,64,0,314110,2062,0,172,4,1417.420,1.280,1045.590,0.940,2930101,1,6,9.600,2,0,10,6,0,0,6,6273.560


In [22]:
df_ind_trans = df_ind_trans.rename(columns={'index': 'id'})

In [23]:
df_ind_trans.columns

Index(['id', 'CBO_Ocupação_2002', 'Faixa_Etária', 'Faixa_Hora_Contrat',
       'Faixa_Remun_Dezem_(SM)', 'Faixa_Remun_Média_(SM)',
       'Faixa_Tempo_Emprego', 'Escolaridade_após_2005', 'Qtd_Hora_Contr',
       'Idade', 'Ind_Simples', 'Município', 'Natureza_Jurídica',
       'Ind_Portador_Defic', 'Qtd_Dias_Afastamento', 'Raça_Cor',
       'Vl_Remun_Dezembro_Nom', 'Vl_Remun_Dezembro_(SM)', 'Vl_Remun_Média_Nom',
       'Vl_Remun_Média_(SM)', 'CNAE_2.0_Subclasse', 'Sexo_Trabalhador',
       'Tamanho_Estabelecimento', 'Tempo_Emprego', 'Tipo_Admissão',
       'Tipo_Defic', 'Tipo_Vínculo', 'IBGE_Subsetor', 'Ind_Trab_Intermitente',
       'Ind_Trab_Parcial', 'N_0', 'Vlr_Rem_Ano'],
      dtype='object')

In [24]:
df_ind_trans = df_ind_trans[['id', 'CBO_Ocupação_2002', 'Faixa_Etária', 'Faixa_Hora_Contrat',
       'Faixa_Remun_Dezem_(SM)', 'Faixa_Remun_Média_(SM)',
       'Faixa_Tempo_Emprego', 'Escolaridade_após_2005', 'Qtd_Hora_Contr',
       'Idade', 'Ind_Simples', 'Município', 'Natureza_Jurídica',
       'Ind_Portador_Defic', 'Qtd_Dias_Afastamento', 'Raça_Cor',
       'Vl_Remun_Dezembro_Nom', 
        'CNAE_2.0_Subclasse', 'Sexo_Trabalhador',
       'Tamanho_Estabelecimento', 'Tempo_Emprego', 'Tipo_Admissão',
       'Tipo_Defic', 'Tipo_Vínculo', 'IBGE_Subsetor', 'Ind_Trab_Intermitente',
       'Ind_Trab_Parcial']]

### Entendimento das variaveis ###

In [25]:
def unique_values(df):
    all_cols = df.columns
    feature_name = []
    num_unique_val = []
    name_unique_val = []
    datatype = []
    
    for col in all_cols:
        feature_name.append(col)
        num_unique_val.append(df[col].nunique())
        name_unique_val.append(df[col].unique())
        datatype.append(df[col].dtype)
    return pd.DataFrame({'Feature Name': feature_name, 'Number of Unique Value': num_unique_val, 'Name of Unique Values': name_unique_val,'datatype': datatype})

In [26]:
unique_values(df_ind_trans)

,Feature Name,Number of Unique Value,Name of Unique Values,datatype
0,id,1146857,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15,...",int64
1,CBO_Ocupação_2002,1962,"[782310, 821205, 811770, 514320, 514325, 78321...",int32
2,Faixa_Etária,8,"[7, 8, 4, 5, 6, 3, 2, 1]",int64
3,Faixa_Hora_Contrat,7,"[6, 1, 5, 3, 4, 99, 2]",int64
4,Faixa_Remun_Dezem_(SM),12,"[3, 5, 4, 2, 9, 1, 6, 7, 10, 8, 12, 11]",int64
5,Faixa_Remun_Média_(SM),12,"[2, 3, 1, 4, 8, 5, 6, 9, 0, 7, 11, 10]",int64
6,Faixa_Tempo_Emprego,9,"[3, 6, 4, 2, 5, 1, 8, 7, 99]",int64
7,Escolaridade_após_2005,11,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",int64
8,Qtd_Hora_Contr,45,"[44, 1, 36, 39, 20, 40, 22, 43, 41, 38, 32, 21...",int64
9,Idade,81,"[50, 61, 68, 28, 64, 34, 49, 26, 27, 18, 58, 4...",int64


In [27]:
a = df_ind_trans.groupby(['Natureza_Jurídica']).agg(N = ('id', 'count')).reset_index()
a = a.sort_values('N', ascending=False)
a = a[a['N'] > 100]
lst_Natureza_Jurídica = a['Natureza_Jurídica'].unique().tolist()
lst_Natureza_Jurídica

[2062, 2054, 2305, 2135, 2046, 2038, 2143, 2011, 2240, 2313, 4081, 2151, 4120]

In [28]:
DF_Municipios = pd.read_excel('Municipio por mesorregião.xlsx')
DF_Municipios.head()

,Uf,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Cod Municipio Completo,Nome_Município,Mesorregião Geográfica.1,Nome_Mesorregião.1,Nome_UF.1,Cod Municipio 6
0,11,Rondônia,1102,Leste Rondoniense,11004,Ji-Paraná,1435,1101435,Nova União,1102,Leste Rondoniense,Rondônia,110143
1,11,Rondônia,1102,Leste Rondoniense,11004,Ji-Paraná,1559,1101559,Teixeirópolis,1102,Leste Rondoniense,Rondônia,110155
2,11,Rondônia,1102,Leste Rondoniense,11004,Ji-Paraná,1609,1101609,Theobroma,1102,Leste Rondoniense,Rondônia,110160
3,11,Rondônia,1102,Leste Rondoniense,11004,Ji-Paraná,1708,1101708,Urupá,1102,Leste Rondoniense,Rondônia,110170
4,11,Rondônia,1102,Leste Rondoniense,11004,Ji-Paraná,1807,1101807,Vale do Paraíso,1102,Leste Rondoniense,Rondônia,110180


In [29]:
DF_Municipios['Nome_UF'].unique().tolist()

['Rondônia',
 'Acre',
 'Amazonas',
 'Roraima',
 'Pará',
 'Amapá',
 'Tocantins',
 'Maranhão',
 'Piauí',
 'Ceará',
 'Rio Grande do Norte',
 'Paraíba',
 'Pernambuco',
 'Alagoas',
 'Sergipe',
 'Bahia',
 'Minas Gerais',
 'Espírito Santo',
 'Rio de Janeiro',
 'São Paulo',
 'Paraná',
 'Santa Catarina',
 'Rio Grande do Sul',
 'Mato Grosso do Sul',
 'Mato Grosso',
 'Goiás',
 'Distrito Federal']

In [30]:
mappings = {
 'Rondônia': 'RO',
 'Acre': 'AC',
 'Amazonas': 'AM',
 'Roraima': 'RR',
 'Pará': 'PA',
 'Amapá': 'AP',
 'Tocantins': 'TO',
 'Maranhão': 'MA',
 'Piauí': 'PI',
 'Ceará': 'CE',
 'Rio Grande do Norte': 'RN',
 'Paraíba': 'PB',
 'Pernambuco': 'PE',
 'Alagoas': 'AL',
 'Sergipe': 'SE',
 'Bahia': 'BA',
 'Minas Gerais': 'MG',
 'Espírito Santo': 'ES',
 'Rio de Janeiro': 'RJ',
 'São Paulo': 'SP',
 'Paraná': 'PR',
 'Santa Catarina': 'SC',
 'Rio Grande do Sul': 'RS',
 'Mato Grosso do Sul': 'MS',
 'Mato Grosso': 'MT',
 'Goiás': 'GO',
 'Distrito Federal': 'DF'
}

In [31]:
DF_Municipios['UF'] = DF_Municipios['Nome_UF'].map(mappings)
DF_Municipios.head()

,Uf,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Cod Municipio Completo,Nome_Município,Mesorregião Geográfica.1,Nome_Mesorregião.1,Nome_UF.1,Cod Municipio 6,UF
0,11,Rondônia,1102,Leste Rondoniense,11004,Ji-Paraná,1435,1101435,Nova União,1102,Leste Rondoniense,Rondônia,110143,RO
1,11,Rondônia,1102,Leste Rondoniense,11004,Ji-Paraná,1559,1101559,Teixeirópolis,1102,Leste Rondoniense,Rondônia,110155,RO
2,11,Rondônia,1102,Leste Rondoniense,11004,Ji-Paraná,1609,1101609,Theobroma,1102,Leste Rondoniense,Rondônia,110160,RO
3,11,Rondônia,1102,Leste Rondoniense,11004,Ji-Paraná,1708,1101708,Urupá,1102,Leste Rondoniense,Rondônia,110170,RO
4,11,Rondônia,1102,Leste Rondoniense,11004,Ji-Paraná,1807,1101807,Vale do Paraíso,1102,Leste Rondoniense,Rondônia,110180,RO


In [32]:
DF_Municipios = DF_Municipios[['Cod Municipio 6', 'UF', 'Nome_Município']]
DF_Municipios = DF_Municipios.rename(columns={'Cod Municipio 6': 'Município'})
df_ind_trans = df_ind_trans.merge(DF_Municipios, on=['Município'], how='left')

In [33]:
df_ind_trans.head()

,id,CBO_Ocupação_2002,Faixa_Etária,Faixa_Hora_Contrat,Faixa_Remun_Dezem_(SM),Faixa_Remun_Média_(SM),Faixa_Tempo_Emprego,Escolaridade_após_2005,Qtd_Hora_Contr,Idade,Ind_Simples,Município,Natureza_Jurídica,Ind_Portador_Defic,Qtd_Dias_Afastamento,Raça_Cor,Vl_Remun_Dezembro_Nom,CNAE_2.0_Subclasse,Sexo_Trabalhador,Tamanho_Estabelecimento,Tempo_Emprego,Tipo_Admissão,Tipo_Defic,Tipo_Vínculo,IBGE_Subsetor,Ind_Trab_Intermitente,Ind_Trab_Parcial,UF,Nome_Município
0,0,782310,7,6,3,2,3,1,44,50,1,314800,2062,0,0,4,1372.730,3101200,1,4,7.300,2,0,10,7,0,0,MG,Patos de Minas
1,1,821205,7,1,3,3,6,1,1,61,0,316720,2305,0,20,2,1340.190,2411300,1,7,37.600,0,0,10,3,0,0,MG,Sete Lagoas
2,2,821205,8,5,5,3,6,1,36,68,0,316720,2305,0,62,8,2263.800,2411300,1,7,38.900,0,0,10,3,0,0,MG,Sete Lagoas
3,3,811770,4,6,4,2,3,1,44,28,1,314520,2135,0,0,2,2083.360,1533500,1,3,6.100,2,0,10,12,0,0,MG,Nova Serrana
4,4,514320,7,6,3,1,3,1,44,64,0,314110,2062,0,172,4,1417.420,2930101,1,6,9.600,2,0,10,6,0,0,MG,Matozinhos


## Preparing Data ##

In [34]:
df_ind_trans.columns

Index(['id', 'CBO_Ocupação_2002', 'Faixa_Etária', 'Faixa_Hora_Contrat',
       'Faixa_Remun_Dezem_(SM)', 'Faixa_Remun_Média_(SM)',
       'Faixa_Tempo_Emprego', 'Escolaridade_após_2005', 'Qtd_Hora_Contr',
       'Idade', 'Ind_Simples', 'Município', 'Natureza_Jurídica',
       'Ind_Portador_Defic', 'Qtd_Dias_Afastamento', 'Raça_Cor',
       'Vl_Remun_Dezembro_Nom', 'CNAE_2.0_Subclasse', 'Sexo_Trabalhador',
       'Tamanho_Estabelecimento', 'Tempo_Emprego', 'Tipo_Admissão',
       'Tipo_Defic', 'Tipo_Vínculo', 'IBGE_Subsetor', 'Ind_Trab_Intermitente',
       'Ind_Trab_Parcial', 'UF', 'Nome_Município'],
      dtype='object')

In [35]:
df_ind_trans['Tipo_Vínculo'] = df_ind_trans['Tipo_Vínculo'].astype(str)
df_ind_trans['Tipo_Vínculo'] = 'Tipo_Vínculo_' + df_ind_trans['Tipo_Vínculo']
df_ind_trans['Tipo_Vínculo'].head()

0    Tipo_Vínculo_10
1    Tipo_Vínculo_10
2    Tipo_Vínculo_10
3    Tipo_Vínculo_10
4    Tipo_Vínculo_10
Name: Tipo_Vínculo, dtype: object

In [36]:
df_ind_trans['CBO_Ocupação_2002'] = df_ind_trans['CBO_Ocupação_2002'].replace('-', '', regex=True)
df_ind_trans['CBO_Ocupação_2002'] = df_ind_trans['CBO_Ocupação_2002'].astype(int)
df_ind_trans['CBO'] = round(((df_ind_trans['CBO_Ocupação_2002']/100) - ((df_ind_trans['CBO_Ocupação_2002']%100)/100)), 2)
df_ind_trans['CBO'] = df_ind_trans['CBO'].astype(int)
df_ind_trans['CBO'] = df_ind_trans['CBO'].astype(str)
df_ind_trans['CBO'] = 'EC_' + df_ind_trans['CBO']
df_ind_trans['CBO'].head()

0    EC_7823
1    EC_8212
2    EC_8212
3    EC_8117
4    EC_5143
Name: CBO, dtype: object

In [37]:
df_ind_trans['CNAE_2.0_Subclasse'] = df_ind_trans['CNAE_2.0_Subclasse'].astype(str)
df_ind_trans['CNAE_2.0_Subclasse'] = 'CNAE_' + df_ind_trans['CNAE_2.0_Subclasse']
df_ind_trans['CNAE_2.0_Subclasse'].head()

0    CNAE_3101200
1    CNAE_2411300
2    CNAE_2411300
3    CNAE_1533500
4    CNAE_2930101
Name: CNAE_2.0_Subclasse, dtype: object

In [38]:
df_ind_trans['UF'] = df_ind_trans['UF'].str.lower()
df_ind_trans['Nome_Município'] = df_ind_trans['Nome_Município'].str.lower().replace(' ', '', regex=True)
df_ind_trans['cidades'] = df_ind_trans['UF'] + "_" + df_ind_trans['Nome_Município']
df_ind_trans[['UF', 'Nome_Município', 'cidades']].head()

,UF,Nome_Município,cidades
0,mg,patosdeminas,mg_patosdeminas
1,mg,setelagoas,mg_setelagoas
2,mg,setelagoas,mg_setelagoas
3,mg,novaserrana,mg_novaserrana
4,mg,matozinhos,mg_matozinhos


In [39]:
df_ind_trans['IBGE_Subsetor'].value_counts()

IBGE_Subsetor
13    259646
3     174266
10    136764
11    134380
2      87812
4      83899
6      74796
7      56435
9      39225
8      34822
5      34584
12     27430
21      2628
19       163
20         7
Name: count, dtype: int64

In [40]:
df_ind_trans['IBGE_Subsetor'] = df_ind_trans['IBGE_Subsetor'].astype(str)
df_ind_trans['IBGE_Subsetor'] = 'IBGE_Subsetor_' + df_ind_trans['IBGE_Subsetor']
df_ind_trans['IBGE_Subsetor'].head()

0     IBGE_Subsetor_7
1     IBGE_Subsetor_3
2     IBGE_Subsetor_3
3    IBGE_Subsetor_12
4     IBGE_Subsetor_6
Name: IBGE_Subsetor, dtype: object

In [41]:
# categorical columns without an order :

a = unique_values(df_ind_trans)
a = a[(a['datatype'] == 'object')]
nominal_columns = a['Feature Name'].tolist()
nominal_columns

['CNAE_2.0_Subclasse',
 'Tipo_Vínculo',
 'IBGE_Subsetor',
 'UF',
 'Nome_Município',
 'CBO',
 'cidades']

In [42]:
for Names in nominal_columns: 
    print(Names, len(df_ind_trans[Names].unique()))

CNAE_2.0_Subclasse 405
Tipo_Vínculo 15
IBGE_Subsetor 15
UF 3
Nome_Município 945
CBO 571
cidades 948


In [43]:
nominal_columns_new = nominal_columns.copy()
for i in range(0, len(nominal_columns_new)):
    nominal_columns_new[i] = nominal_columns_new[i] + "_targetencoder_"

nominal_columns_new 

['CNAE_2.0_Subclasse_targetencoder_',
 'Tipo_Vínculo_targetencoder_',
 'IBGE_Subsetor_targetencoder_',
 'UF_targetencoder_',
 'Nome_Município_targetencoder_',
 'CBO_targetencoder_',
 'cidades_targetencoder_']

In [44]:
smoothing = 2550.0
min_samples_leaf = 10
enc_auto = ce.TargetEncoder(smoothing=smoothing, min_samples_leaf=min_samples_leaf, handle_unknown='return_nan', return_df=True)

for Names, Names_New in zip(nominal_columns, nominal_columns_new): 
    df_ind_trans[Names_New] = enc_auto.fit_transform(df_ind_trans[Names], df_ind_trans['Vl_Remun_Dezembro_Nom'])


In [45]:
df_ind_trans.head()

,id,CBO_Ocupação_2002,Faixa_Etária,Faixa_Hora_Contrat,Faixa_Remun_Dezem_(SM),Faixa_Remun_Média_(SM),Faixa_Tempo_Emprego,Escolaridade_após_2005,Qtd_Hora_Contr,Idade,Ind_Simples,Município,Natureza_Jurídica,Ind_Portador_Defic,Qtd_Dias_Afastamento,Raça_Cor,Vl_Remun_Dezembro_Nom,CNAE_2.0_Subclasse,Sexo_Trabalhador,Tamanho_Estabelecimento,Tempo_Emprego,Tipo_Admissão,Tipo_Defic,Tipo_Vínculo,IBGE_Subsetor,Ind_Trab_Intermitente,Ind_Trab_Parcial,UF,Nome_Município,CBO,cidades,CNAE_2.0_Subclasse_targetencoder_,Tipo_Vínculo_targetencoder_,IBGE_Subsetor_targetencoder_,UF_targetencoder_,Nome_Município_targetencoder_,CBO_targetencoder_,cidades_targetencoder_
0,0,782310,7,6,3,2,3,1,44,50,1,314800,2062,0,0,4,1372.730,CNAE_3101200,1,4,7.300,2,0,Tipo_Vínculo_10,IBGE_Subsetor_7,0,0,mg,patosdeminas,EC_7823,mg_patosdeminas,2012.803,3363.495,2012.400,2902.972,2662.986,2455.315,2662.986
1,1,821205,7,1,3,3,6,1,1,61,0,316720,2305,0,20,2,1340.190,CNAE_2411300,1,7,37.600,0,0,Tipo_Vínculo_10,IBGE_Subsetor_3,0,0,mg,setelagoas,EC_8212,mg_setelagoas,3221.220,3363.495,4149.241,2902.972,3183.867,4420.601,3183.867
2,2,821205,8,5,5,3,6,1,36,68,0,316720,2305,0,62,8,2263.800,CNAE_2411300,1,7,38.900,0,0,Tipo_Vínculo_10,IBGE_Subsetor_3,0,0,mg,setelagoas,EC_8212,mg_setelagoas,3221.220,3363.495,4149.241,2902.972,3183.867,4420.601,3183.867
3,3,811770,4,6,4,2,3,1,44,28,1,314520,2135,0,0,2,2083.360,CNAE_1533500,1,3,6.100,2,0,Tipo_Vínculo_10,IBGE_Subsetor_12,0,0,mg,novaserrana,EC_8117,mg_novaserrana,2231.034,3363.495,1877.463,2902.972,1971.877,2210.861,1971.877
4,4,514320,7,6,3,1,3,1,44,64,0,314110,2062,0,172,4,1417.420,CNAE_2930101,1,6,9.600,2,0,Tipo_Vínculo_10,IBGE_Subsetor_6,0,0,mg,matozinhos,EC_5143,mg_matozinhos,3276.079,3363.495,4491.901,2902.972,2785.046,1583.565,2785.046


In [46]:
df_ind_trans.columns

Index(['id', 'CBO_Ocupação_2002', 'Faixa_Etária', 'Faixa_Hora_Contrat',
       'Faixa_Remun_Dezem_(SM)', 'Faixa_Remun_Média_(SM)',
       'Faixa_Tempo_Emprego', 'Escolaridade_após_2005', 'Qtd_Hora_Contr',
       'Idade', 'Ind_Simples', 'Município', 'Natureza_Jurídica',
       'Ind_Portador_Defic', 'Qtd_Dias_Afastamento', 'Raça_Cor',
       'Vl_Remun_Dezembro_Nom', 'CNAE_2.0_Subclasse', 'Sexo_Trabalhador',
       'Tamanho_Estabelecimento', 'Tempo_Emprego', 'Tipo_Admissão',
       'Tipo_Defic', 'Tipo_Vínculo', 'IBGE_Subsetor', 'Ind_Trab_Intermitente',
       'Ind_Trab_Parcial', 'UF', 'Nome_Município', 'CBO', 'cidades',
       'CNAE_2.0_Subclasse_targetencoder_', 'Tipo_Vínculo_targetencoder_',
       'IBGE_Subsetor_targetencoder_', 'UF_targetencoder_',
       'Nome_Município_targetencoder_', 'CBO_targetencoder_',
       'cidades_targetencoder_'],
      dtype='object')

In [47]:
df_ind_trans = df_ind_trans[['id', 'Faixa_Etária', 'Faixa_Hora_Contrat', 'Faixa_Tempo_Emprego',
       'Escolaridade_após_2005', 'Qtd_Hora_Contr', 'Idade', 'Ind_Simples',
        'Ind_Portador_Defic',
       'Qtd_Dias_Afastamento', 'Raça_Cor', 'Vl_Remun_Dezembro_Nom',
        'Sexo_Trabalhador', 'Tamanho_Estabelecimento',
       'Tempo_Emprego', 'Tipo_Admissão', 'Tipo_Defic', 
        'Ind_Trab_Intermitente', 'Ind_Trab_Parcial',  'CNAE_2.0_Subclasse_targetencoder_',
       'Tipo_Vínculo_targetencoder_', 'IBGE_Subsetor_targetencoder_',
       'CBO_targetencoder_', 'cidades_targetencoder_']]

In [48]:
df_ind_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1146857 entries, 0 to 1146856
Data columns (total 24 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   id                                 1146857 non-null  int64  
 1   Faixa_Etária                       1146857 non-null  int64  
 2   Faixa_Hora_Contrat                 1146857 non-null  int64  
 3   Faixa_Tempo_Emprego                1146857 non-null  int64  
 4   Escolaridade_após_2005             1146857 non-null  int64  
 5   Qtd_Hora_Contr                     1146857 non-null  int64  
 6   Idade                              1146857 non-null  int64  
 7   Ind_Simples                        1146857 non-null  int64  
 8   Ind_Portador_Defic                 1146857 non-null  int64  
 9   Qtd_Dias_Afastamento               1146857 non-null  int64  
 10  Raça_Cor                           1146857 non-null  int64  
 11  Vl_Remun_Dezembro_Nom   

In [49]:
df_ind_trans.describe().T

,count,mean,std,min,25%,50%,75%,max
id,1146857.000,612543.207,350884.522,0.000,313180.000,610969.000,910740.000,1223050.000
Faixa_Etária,1146857.000,5.034,1.301,1.000,4.000,5.000,6.000,8.000
Faixa_Hora_Contrat,1146857.000,6.799,9.459,1.000,6.000,6.000,6.000,99.000
Faixa_Tempo_Emprego,1146857.000,4.899,2.456,1.000,3.000,5.000,7.000,99.000
Escolaridade_após_2005,1146857.000,6.573,1.538,1.000,6.000,7.000,7.000,11.000
Qtd_Hora_Contr,1146857.000,42.299,6.115,0.000,44.000,44.000,44.000,44.000
Idade,1146857.000,36.367,11.258,14.000,27.000,35.000,44.000,98.000
Ind_Simples,1146857.000,0.283,0.451,0.000,0.000,0.000,1.000,1.000
Ind_Portador_Defic,1146857.000,0.016,0.125,0.000,0.000,0.000,0.000,1.000
Qtd_Dias_Afastamento,1146857.000,7.794,29.424,0.000,0.000,0.000,1.000,365.000


In [50]:
df_ind_trans.to_csv("DF_Industria_tranformacao_2021.csv", sep=';', index=False)